In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
os.chdir("/content/gdrive/My Drive/Colab Notebooks")
os.getcwd()

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


'/content/gdrive/My Drive/Colab Notebooks'

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [ ]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

from sklearn.compose import ColumnTransformer

onehotencoder = ColumnTransformer([("Country", OneHotEncoder(), [1])], remainder = 'passthrough')
X = onehotencoder.fit_transform(X)
X = X[:,1:]

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

#Initializing the ANN
classifier = Sequential()

#input layer and the first hidden layer
classifier.add(Dense(6, activation='relu', kernel_initializer='he_uniform', input_shape=(11,) ))
classifier.add(Dropout(0.1))

#add second hidden layer
classifier.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
classifier.add(Dropout(0.1))

#add output layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'))


In [ ]:
#compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
#fit model
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
800/800 [==============================] - 1s 965us/step - loss: 0.6135 - accuracy: 0.7055
Epoch 2/100
800/800 [==============================] - 1s 990us/step - loss: 0.4936 - accuracy: 0.7910
Epoch 3/100
800/800 [==============================] - 1s 970us/step - loss: 0.4671 - accuracy: 0.8002
Epoch 4/100
800/800 [==============================] - 1s 953us/step - loss: 0.4550 - accuracy: 0.8011
Epoch 5/100
800/800 [==============================] - 1s 942us/step - loss: 0.4400 - accuracy: 0.8050
Epoch 6/100
800/800 [==============================] - 1s 974us/step - loss: 0.4401 - accuracy: 0.8066
Epoch 7/100
800/800 [==============================] - 1s 963us/step - loss: 0.4319 - accuracy: 0.8101
Epoch 8/100
800/800 [==============================] - 1s 926us/step - loss: 0.4223 - accuracy: 0.8160
Epoch 9/100
800/800 [==============================] - 1s 917us/step - loss: 0.4223 - accuracy: 0.8158
Epoch 10/100
800/800 [==============================] - 1s 941us/step - l

In [ ]:
#predict the result
y_pred = classifier.predict(X_test)
y_pred = (y_pred >0.5)

In [ ]:
#comfusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[1551   44]
 [ 225  180]]


In [ ]:
#Evaulte the ANN (K-fold)
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
  #Initializing the ANN
  classifier = Sequential()

  #input layer and the first hidden layer
  classifier.add(Dense(6, activation='relu', kernel_initializer='he_uniform', input_shape=(11,) ))
  classifier.add(Dropout(0.1))

  #add second hidden layer
  classifier.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
  classifier.add(Dropout(0.1))

  #add output layer
  classifier.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'))
  
  #compiling the ANN
  classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size=10, epochs=100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv=10)

Epoch 1/100
720/720 [==============================] - 1s 940us/step - loss: 0.6101 - accuracy: 0.6882
Epoch 2/100
720/720 [==============================] - 1s 1ms/step - loss: 0.4770 - accuracy: 0.7968
Epoch 3/100
720/720 [==============================] - 1s 930us/step - loss: 0.4506 - accuracy: 0.8006
Epoch 4/100
720/720 [==============================] - 1s 896us/step - loss: 0.4433 - accuracy: 0.8097
Epoch 5/100
720/720 [==============================] - 1s 966us/step - loss: 0.4305 - accuracy: 0.8129
Epoch 6/100
720/720 [==============================] - 1s 1ms/step - loss: 0.4256 - accuracy: 0.8163
Epoch 7/100
720/720 [==============================] - 1s 925us/step - loss: 0.4242 - accuracy: 0.8165
Epoch 8/100
720/720 [==============================] - 1s 994us/step - loss: 0.4185 - accuracy: 0.8161
Epoch 9/100
720/720 [==============================] - 1s 1ms/step - loss: 0.4173 - accuracy: 0.8163
Epoch 10/100
720/720 [==============================] - 1s 933us/step - loss: 0

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

0.8537499964237213
0.011524419581068826


In [ ]:
#Tuning the ANN
from sklearn.model_selection import GridSearchCV
def build_classifier1(optimizer):
  #Initializing the ANN
  classifier = Sequential()

  #input layer and the first hidden layer
  classifier.add(Dense(6, activation='relu', kernel_initializer='he_uniform', input_shape=(11,) ))
  classifier.add(Dropout(0.1))

  #add second hidden layer
  classifier.add(Dense(6, activation='relu', kernel_initializer='he_uniform'))
  classifier.add(Dropout(0.1))

  #add output layer
  classifier.add(Dense(1, activation='sigmoid', kernel_initializer='he_uniform'))
  
  #compiling the ANN
  classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

  return classifier

classifier = KerasClassifier(build_fn = build_classifier1)
parameters = {'batch_size':[25, 32], 'epochs':[100, 500], 'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)
grid_search = grid_search.fit(X_train,y_train)
best_parameter = grid_search.best_params_
best_accuracy = grid_search.best_score_

串流輸出內容已截斷至最後 5000 行。
225/225 [==============================] - 0s 984us/step - loss: 0.6672 - accuracy: 0.6478
Epoch 2/500
225/225 [==============================] - 0s 988us/step - loss: 0.5471 - accuracy: 0.7919
Epoch 3/500
225/225 [==============================] - 0s 959us/step - loss: 0.4939 - accuracy: 0.7944
Epoch 4/500
225/225 [==============================] - 0s 1ms/step - loss: 0.4693 - accuracy: 0.7967
Epoch 5/500
225/225 [==============================] - 0s 1ms/step - loss: 0.4514 - accuracy: 0.7969
Epoch 6/500
225/225 [==============================] - 0s 1ms/step - loss: 0.4448 - accuracy: 0.7979
Epoch 7/500
225/225 [==============================] - 0s 1ms/step - loss: 0.4365 - accuracy: 0.8057
Epoch 8/500
225/225 [==============================] - 0s 1ms/step - loss: 0.4348 - accuracy: 0.8125
Epoch 9/500
225/225 [==============================] - 0s 1ms/step - loss: 0.4309 - accuracy: 0.8119
Epoch 10/500
225/225 [==============================] - 0s 1ms/step - loss: 

In [ ]:
print(best_parameter)
print(best_accuracy)

{'batch_size': 32, 'epochs': 500, 'optimizer': 'rmsprop'}
0.8587499999999999
